In [1]:
import torch
import torch.nn as nn
import numpy as np

from torch.utils.data import DataLoader,TensorDataset
from torch.optim import RMSprop
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [2]:
EPOCHS=5

In [3]:
features,target = make_regression(n_features=10,
                                  n_samples=1000)
features_train,features_test,target_train,target_test = train_test_split(features,
                                                                         target,
                                                                         test_size=0.1,
                                                                         random_state=1)
torch.manual_seed(0)
np.random.seed(0)

x_train = torch.from_numpy(features_train).float()
y_train = torch.from_numpy(target_train).float().view(-1,1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1,1)

In [4]:
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet,self).__init__()
        self.sequential = torch.nn.Sequential(
            torch.nn.Linear(10,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,16),
            torch.nn.ReLU(),
            torch.nn.Linear(16,1)
        )
    
    def forward(self,x):
        x = self.sequential(x)
        return(x)

In [5]:
network = SimpleNeuralNet()

criterion = nn.MSELoss()
optimizer = RMSprop(network.parameters())

train_data = TensorDataset(x_train,y_train)
train_loader = DataLoader(train_data,batch_size=100,shuffle=True)

for epoch in range(EPOCHS):
    for batch_idx,(data,target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
    print("Epoch:", epoch+1,"\tLoss:", loss.item())

with torch.no_grad():
    output = network(x_test)
    test_loss = float(criterion(output,y_test))
    print("Test MSE:" , test_loss)

Epoch: 1 	Loss: 12613.5087890625
Epoch: 2 	Loss: 2104.848388671875
Epoch: 3 	Loss: 554.7191162109375
Epoch: 4 	Loss: 344.9694519042969
Epoch: 5 	Loss: 309.48065185546875
Test MSE: 241.0288543701172
